In [441]:
import pandas as pd
import numpy as np
import random as rnd
import datetime
from tqdm import tqdm

'2015-08-07'

In [442]:
#количество пользователей
num=101
# таблица с тарифами
tarifs = pd.DataFrame(columns=['Name', 'Date_start', 'Date_end', 'Minutes', 'Sms', 'Mb'])
tarifs = tarifs.append({'Name': 'БезПереплат', 'Date_start': (datetime.datetime.now() - datetime.timedelta(days=180+1200+rnd.randrange(0, 1200, 1))).strftime('%Y-%m-%d'), 'Date_end': (datetime.datetime.now() + datetime.timedelta(days=rnd.randrange(0, 1200, 1))).strftime('%Y-%m-%d'), 'Minutes': 300, 'Sms': 150, 'Mb': 1024}, ignore_index=True)
tarifs = tarifs.append({'Name': 'Максимум', 'Date_start': (datetime.datetime.now() - datetime.timedelta(days=180+1200+rnd.randrange(0, 1200, 1))).strftime('%Y-%m-%d'), 'Date_end': (datetime.datetime.now() + datetime.timedelta(days=rnd.randrange(0, 1200, 1))).strftime('%Y-%m-%d'), 'Minutes': 800, 'Sms': 500, 'Mb': 16384}, ignore_index=True)
tarifs = tarifs.append({'Name': 'СреднийБит', 'Date_start': (datetime.datetime.now() - datetime.timedelta(days=180+1200+rnd.randrange(0, 1200, 1))).strftime('%Y-%m-%d'), 'Date_end': (datetime.datetime.now() + datetime.timedelta(days=rnd.randrange(0, 1200, 1))).strftime('%Y-%m-%d'), 'Minutes': 550, 'Sms': 300, 'Mb': 1024*4}, ignore_index=True)

In [460]:
#генераторы таблиц с пользователями и событиями
def table1_generator():
    i = 0
    users = pd.DataFrame(columns=['Balance', 'Date_of_create', 'Ages', 'City', 'Last_activity', 'Tarif_ID'])
    for i in tqdm(range(num)):
        users = users.append({'Balance': (rnd.randrange(0, 1001, 1)), 
                              'Date_of_create': (datetime.datetime.now() - datetime.timedelta(days=1000+rnd.randrange(0, 1200, 1))).strftime('%Y-%m-%d'),
                              'Ages': rnd.randrange(18, 61, 1), 
                              'City': random.choice(['Москва', 'Санкт-Петербург', 'Самара']), 
                              'Last_activity': (datetime.datetime.now() - datetime.timedelta(seconds=rnd.randrange(0, 10000000, 1))).strftime('%Y-%m-%d %H:%M:%S'), 
                              'Tarif_ID': rnd.randrange(0, 3, 1)}, ignore_index=True)
    return users

def table2_generator(rows):
    i = 0
    users = pd.DataFrame(columns=['Time', 'User_ID', 'Type', 'num'])
    for i in tqdm(range(rows)):
        users = users.append({'Time': (datetime.datetime.now() - datetime.timedelta(seconds=rnd.randrange(0, 3000000, 1))).strftime('%Y-%m-%d %H:%M:%S'), 
                              'User_ID': rnd.randrange(0, 101, 1),
                              'Type': random.choice(['Телефон', 'СМС', 'Интернет']), 
                              'num': rnd.randrange(0, 300, 1)}, ignore_index=True)
        i+=1
    return users

In [461]:
#генерация
users=table1_generator()
all_rows=table2_generator(10000)

100%|██████████| 10000/10000 [00:52<00:00, 189.26it/s]


,Time,User_ID,Type,num
0,2022-02-27 03:47:59,31,Интернет,159
1,2022-03-11 01:56:39,22,Интернет,169
2,2022-03-14 21:21:11,16,Телефон,94
3,2022-03-09 03:54:53,63,СМС,106
4,2022-03-27 15:39:33,12,СМС,44
...,...,...,...,...
9995,2022-03-15 00:29:45,53,Телефон,218
9996,2022-03-18 19:27:16,94,Телефон,68
9997,2022-03-22 21:50:15,19,Интернет,52
9998,2022-03-22 17:10:50,56,СМС,186


In [463]:
#перевод времени в формат YYMMDD
all_rows['Time']=pd.to_datetime(all_rows.Time)
all_rows['Time_YMD'] = all_rows['Time'].dt.strftime('%m-%d-%Y')

In [465]:
#итоговая обработка
all_rows[['User_ID', 'Time_YMD']].join(all_rows.pivot(columns = 'Type', values = 'num')).groupby(['User_ID', 'Time_YMD']).sum()

Интернет  СМС  Телефон
User_ID Time_YMD                          
0       02-22-2022         0  103        0
        02-23-2022        86  331      132
        02-24-2022       210  453      268
        02-25-2022       105    0      345
        02-27-2022       210  236      238
...                      ...  ...      ...
100     03-23-2022         0  486      148
        03-24-2022        51  688      158
        03-25-2022       269  437        0
        03-26-2022       297  405        0
        03-28-2022        95  295        0

[3342 rows x 3 columns]